<h1><center>TP2 - Ex.1</center></h1>
<p><center>Abril 1, 2024</center></p>


### Estruturas Criptográficas

PG53886, Ivo Miguel Alves Ribeiro

A95323, Henrique Ribeiro Fernandes

Estes problemas destinam à iniciação do uso do SageMath  em protótipos de esquemas clássicos de chave pública.


1. Construir uma classe Python que implemente o  EdDSA a partir do “standard” FIPS186-5
    1. A implementação deve conter funções para assinar digitalmente e verificar a assinatura.
    2. A implementação da classe deve usar  uma das “Twisted Edwards Curves” definidas no standard e escolhida  na iniciação da classe: a curva  “edwards25519” ou “edwards448”.

In [2]:
import hashlib
from hashlib import sha512
from sage.all import GF, power_mod
import random

def RGB(key_size, security_strength):
    # Gera uma lista de bits com valores aleatórios (0 ou 1)
    return ''.join(str(random.randint(0, 1)) for _ in range(key_size))


class EdDSA:
    def __init__(self, curve_type):
        if curve_type == "edwards25519":
            self.p = 2^255 - 19
            self.K = GF(self.p)
            self.a = -1
            self.d = self.K(-121665/121666)
            self.key_size = 256
            self.security_strength = 128
            self.c = 3
            self.n = 254
            self.B = (15112221349535400772501151409588531511454012693041857206046113283949847762202,
                      46316835694926478169428394003475163141307993866256225615783033603165251855960)
            self.L = 2**252 + 27742317777372353535851937790883648493
        elif curve_type == "edwards448":
            self.p = pow(2,448) - pow(2,224) - 1
            self.K = GF(self.p)
            self.c = 2
            self.n = 447
            self.d = -39081
            self.a = 1
            self.B = (224580040295924300187604334099896036246789641632564134246125461686950415467406032909029192869357953282578032075146446173674602635247710, 
                      298819210078481492676017930443930673437544040154080242095928241372331506189835876003536878655418784733982303233503462500531545062832660)                                                 
            self.L =  pow(2,446) - 13818066809895115352007386748515426880336692474882178609894547503885
            self.key_size = 456
            self.security_strength = 224
        else:
            raise ValueError("Invalid curve. Supported curves: 'edwards25519' or 'edwards448'")


    def pointAddition(self, P, Q):
        #twisted edwards  curve: (a*x^2 + y^2) mod mod = (1 + d*x^2*y^2) mod mod
        x1 = P[0]; y1 = P[1]; x2 = Q[0]; y2 = Q[1]
        
        x3 = ((x1*y2 + y1*x2) / (1 + self.d*x1*x2*y1*y2)) % self.p
        y3 = ((y1*y2 - self.a*x1*x2) / (1 - self.d*x1*x2*y1*y2)) % self.p
        
        assert (self.a*x3*x3 + y3*y3) % self.p == (1 + self.d*x3*x3*y3*y3) % self.p
        return x3, y3
    

    def applyDoubleAndAddMethod(self, P, k):
        # Pk = P x k
        additionPoint = (P[0], P[1])
        kAsBinary = bin(k)[2:] 
        for i in range(1, len(kAsBinary)):
            currentBit = kAsBinary[i: i+1]
            #always apply doubling
            additionPoint = self.pointAddition(additionPoint, additionPoint)
            if currentBit == '1':
                #add base point
                additionPoint = self.pointAddition(additionPoint, P)
        return additionPoint
    

    def computeG(self, dP):
        return  self.applyDoubleAndAddMethod(self.B, dP)


    def encode_integer(S, b):
        # Convert integer S to little-endian binary string of length b
        return format(S, '0' + str(b) + 'b')[::-1]


    def encode_curve_element(self, x, y):
        x = int(x)
        y = int(y)
        if not (0 <= self.p-x and 0 <= self.p-y):
            raise ValueError("Point coordinates must be within the range 0 ≤ x, y < p")
        # Encode the y-coordinate
        y_encoded = format(y, '0' + str(self.key_size - 1) + 'b')

        # Encode the sign of x (1 if odd, 0 if even)
        x_sign = 1 if x%2 != 0 else 0
        
        return (y_encoded + str(x_sign)).encode()
    

    def decode_curve_element(self, encoded_str):
        encoded_str = encoded_str.decode()
        # Extrair a parte do componente y
        y_encoded = encoded_str[:self.key_size-1]
        # Extrair a sinalização do componente x
        x_sign = int(encoded_str[self.key_size-1])
        # Reconstruir o valor do componente y
        y = int(y_encoded, 2)
        # Step 2: Recover the x-coordinate using the curve equation
        u = y*y - 1
        v = self.d*y*y - self.a
        if self.key_size == 256:
            uv = u/v
            exp = (self.p+3)/8
            w = power_mod(uv, exp, self.p) 
        
            if (v*w*w) % self.p == u % self.p:
                x_final = w
            elif (v*w*w) % self.p == -u % self.p:
                x_final = (w * power_mod(2, (self.p-1)/4, self.p)) % self.p
            else:
                raise ValueError("decoding fails.")
        elif self.key_size == 456:
            uv = u/v
            exp = (self.p+1)/4
            w = power_mod(uv, exp, self.p) 

            if (v*w*w) % self.p == u % self.p:
                x_final = w
            else: 
                raise ValueError("decoding fails.")
        if x_final == 0 and x_sign == 1:
            raise ValueError("decoding fails.")
        elif int(x_final) % 2 != x_sign:
            return self.p - x_final, y
        else:
            return x_final, y
       

    def get_s(self, h_pk):
        h_len = len(h_pk)/2
        b = self.key_size
        if b == 256:
            hdigest1 = h_pk[:h_len]
            h1_list = list(hdigest1)
            h1_list[0]  &= 0b00011111
            # Ensure the last bit of the last octet is 0
            h1_list[-1] &= 0b01111110
            # Ensure the second to last bit of the last octet is 1
            h1_list[-1] |= 0b01000000
        elif b == 456:
            hdigest1 = h_pk[:h_len]
            h1_list = list(hdigest1)
            h1_list[0]  &= 0b00111111
            h1_list[-2] |= 0b01000000
            h1_list[-1] &= 0b11111111
            
        else:
            raise ValueError("Unsupported curve")

        # Step 4: Determine an integer s from hdigest1 using little-endian convention
        return int.from_bytes(bytes(h1_list), 'little')
  

    def key_gen(self):
        # Step 1: Generate a private key
        private_key_bits = RGB(self.key_size, self.security_strength)
        # Step 2: Compute the hash of the private key
        if self.key_size == 256:
            h_pk = sha512(private_key_bits.encode()).digest()
        elif self.key_size == 456:
            h_pk = hashlib.shake_256(private_key_bits.encode()).digest(912)
        else:
            raise ValueError("Unsupported hash function")
        # Step 3: Generate the public key
        s = self.get_s(h_pk)
        # Step 5: Compute the point [s]G
        Ec_point = self.computeG(s)
        #x3 = Ec_point[0]
        #y3 = Ec_point[1]
        #print("pk:",(x3,y3))
        public_key_bits =  self.encode_curve_element(Ec_point[0], Ec_point[1])
        
        # Return private key and public key
        return private_key_bits.encode(), public_key_bits
    

    def sign(self, text, private_key, public_key, context=''):
        # Step 1: Compute the hash of the private key
        if self.key_size == 256:
            h_pk = sha512(private_key).digest()
        elif self.key_size == 456:
            h_pk = hashlib.shake_256(private_key).digest(912)
        else:
            raise ValueError("Unsupported hash function")
        # Step 2: Compute the hash of the message and hdigest2
        byte_length = len(h_pk)//2
        hdigest1 = h_pk[:byte_length]
        hdigest2 = h_pk[byte_length:]
        
        if self.key_size == 256:
            r_octet = sha512(hdigest2 + text).digest()
        elif self.key_size == 456:
            context_bytes = context.encode('utf-8')
            dom4_prefix = b"SigEd448" + bytes([0]) + bytes([len(context_bytes)])
            r_octet = hashlib.shake_256(dom4_prefix + context_bytes + hdigest2 + text).digest(912)
        else:
            raise ValueError("Error")
        
        # Step 3: Compute the point [r]G
        r = int.from_bytes(r_octet, 'little')
        Ec_point = self.computeG(r)
        R = self.encode_curve_element(Ec_point[0], Ec_point[1])
        
        # Step 4: Derive s from H(d) as in the key pair generation algorithm
        s = self.get_s(h_pk)
        
        # Step 5: Compute S
        if self.key_size == 256:
            hashed_data = sha512(R + public_key + text).digest()
            t = int.from_bytes(hashed_data, byteorder='little')
            
            # Calcular S
            S = (r + t * s) % self.L
            S = format(S, '0' + str(self.key_size) + 'b').encode()
        elif self.key_size == 456:
            hashed_data = hashlib.shake_256(dom4_prefix + R + public_key + text).digest(128)
            t = int.from_bytes(hashed_data, byteorder='little')
            
            # Calcular S
            S = (r + t * s) % self.L
            S = format(S, '0' + str(self.key_size) + 'b').encode() 
        else:
            raise ValueError("Unsupported curve")
        
        #print("t: ",t)
        #print("R: ", Ec_point)
        #print("s: ",s)
        # Step 6: Return signature
        return R + S
    
    
    def verify(self, text, Sign, publicKey, context=''):
        # Decode first half of signture as a point R
        half_len = len(Sign)/2
        h1_sign = Sign[:half_len]
        h2_sign = Sign[half_len:]
        R = self.decode_curve_element(h1_sign)
        s = int(h2_sign.decode(), 2)
       
        # Decode public key Q into a point
        Q = self.decode_curve_element(publicKey) 
         
        # 2
        if self.key_size == 256:
            hashed_data = sha512(h1_sign + publicKey + text).digest()
            t_int = int.from_bytes(hashed_data, byteorder='little')
        else:
            context_bytes = context.encode('utf-8')
            dom4_prefix = b"SigEd448" + bytes([0]) + bytes([len(context_bytes)])
            hashed_data = hashlib.shake_256(dom4_prefix + h1_sign + publicKey + text).digest(128)
            t_int = int.from_bytes(hashed_data, byteorder='little')
        # 3
        #print("pk:",Q)
        #print("t: ",t_int)
        #print("R: ", R)
        #print("s: ",s)
        # 4 verification
        check_p1 = self.computeG(s)
        check_p2 = self.pointAddition(R, self.applyDoubleAndAddMethod(Q, t_int))
        #print((check_p1[0], check_p1[1]))
        #print((check_p2[0] , check_p2[1]))
        if check_p1[0] == check_p2[0]%self.p and check_p1[1] == check_p2[1]%self.p:
            return True
        else:
            return False

teste para Ed25519

In [3]:
# Ed25519 is a special form of this curve where a = -1,
# d = -121665/121666. 
# It handles over prime fields where p = 2^255 – 19. 
# -x^2 + y^2 (mod 2255 – 19) = 1 – (121665/121666) * x^2 * y^2 (mod 2255 – 19)
# Ed25519, curve 25519
Ed25519 = EdDSA("edwards25519")

Mensagem = b"Ola Mundo"

sk, pk = Ed25519.key_gen()
print("SK: "+str(sk) + " len: "+str(len(sk)))
print("PK: "+str(pk) + " len: "+str(len(pk)))

Sign = Ed25519.sign(Mensagem, sk, pk)
print("Sign: "+str(Sign) + " len: "+str(len(Sign)))


is_valid = Ed25519.verify(Mensagem, Sign, pk)

if is_valid:
    print("Assinatura válida.")
else:
    print("Assinatura inválida.")

SK: b'1011100000010000010101011100010001000001101110010010011010101101100100111001100001110100000000001001001110011001101010010101101101000101011101011111100100011100010011111010000010011110111101100110011000101011101110101111010011100100001110110001001010010110' len: 256
PK: b'1101110101011110100110011111011011110011001110110101011001111011010101110010000011001101001110010101001100101100111100110010110010110000000110011000001111000101111111100001000011110010110100010110010111000011011110111000011010101000001100000110101101011111' len: 256
Sign: b'00111110110110111100110011110001100011011011101111010101111110101110101010010001000111011001101010010110111111101111110001110101011011100000000101000000100001100000100111000101011100000011001101001011011101111010001011111011000111110111001110100000110101110000111010010111001110111001001101011101111010100011101001001101001001111010101011100000001110011001101010001100010110100101001010100110011000010101101111000010000100100100000001110111011110

teste para ED448

In [4]:
# Ed25519 is a special form of this curve where a = -1,
# d = -121665/121666. 
# It handles over prime fields where p = 2^255 – 19. 
# -x^2 + y^2 (mod 2255 – 19) = 1 – (121665/121666) * x^2 * y^2 (mod 2255 – 19)
Ed448 = EdDSA("edwards448")

Mensagem = b"Ola Mundo"
sk, pk = Ed448.key_gen()
print("SK: "+str(sk) + "len: "+str(len(sk)))
print("PK: "+str(pk) + "len: "+str(len(pk)))

Sign = Ed448.sign(Mensagem, sk, pk)
print("Sign: "+str(Sign) + "len: "+str(len(Sign)))


is_valid = (Ed448.verify(Mensagem, Sign, pk))

if is_valid:
    print("Assinatura válida.")
else:
    print("Assinatura inválida.")

SK: b'011000010100000101100110001000001111000111110010110111101011111111001110101001110101001101011111110111110001010001100111000011010000011111000110101011101110011111101000000000011000100000010001111001010001110100100000100100110111000100100100100100001100010011100000111001100001000110010100110011000001010101101101111011010111111010101000101100000001001010100001000010111110001111011111011000101100110101010101101101110011001101100000001000111110011110010111'len: 456
PK: b'000000000110100010100010111101101101011110001110000100110111001111101001000010010000010010010111010101100100110000101011011100000100101001010100000011110101011011111010100111111111100110001000001001100101001001111000000110110011010100110111000011001011000101000110100000100000110010100111100101100100000101001010100110010110010001111011101001111101000110111011001010100101011011001100011111100101101011011111001110011010100110111110001000011000010010001110'len: 456
Sign: b'000000010111111111001010101101110011111011011111